In [1]:
import os
import pandas as pd
import numpy as np
import xgboost as xgb
import sys
pd.set_option('display.max_columns', None)



In [2]:
src_path = os.path.abspath(os.path.join(os.getcwd(), "../../../src"))
sys.path.append(src_path)
from pipeline import ModelPipeline


In [ ]:
xgb_pipeline = ModelPipeline(dataset_path='/Users/abhitay/Developer/Columbia_projects/CAP/dsi-capstone-spring-2025-TD-anti-money-laundering/Code/Abhitay/IBM/Subset Transactions.csv')

xgb_pipeline.run_preprocessing()

xgb_pipeline.df

Running preprocessing pipeline...

Creating unique ids...

Normalizing currency...

Extracting time features...

Adding cyclical encoding to time feats...

...

New feature columns added: degree_centrality_sent_amount, degree_centrality_received_amount, 
pagerank_sent_amount, pagerank_received_amount

Preprocessing completed successfully!

{
    'renamed': True,
    'duplicates_removed': True,
    'unique_ids_created': True,
    'currency_normalized': True,
    'time_features_extracted': True,
    'cyclical_encoded': True,
    'weekend_encoded': True,
    'label_encoded': True,
    'neighbor_context_computed': True,
    'normalized': False
}

,from_bank,to_bank,received_amount,received_currency,sent_amount,sent_currency,payment_type,is_laundering,from_account_id,to_account_id,from_account_idx,to_account_idx,sent_amount_usd,received_amount_usd,hour_of_day,day_of_week,seconds_since_midnight,timestamp_int,timestamp_scaled,day_sin,day_cos,time_of_day_sin,time_of_day_cos,is_weekend,degree_centrality_sent_amount,pagerank_sent_amount,degree_centrality_received_amount,pagerank_received_amount
0,8,8,3697.340000,12,3697.340000,12,5,0,10_8000EBD30,10_8000EBD30,0,0,3.697340e+03,3.697340e+03,0,3,1200,1.661992e+09,1.661992e+09,0.433884,-0.900969,0.087156,0.996195,0,0.000139,0.000050,0.000139,0.000050
1,109,0,0.010000,12,0.010000,12,3,0,3208_8000F4580,1_8000F5340,1,81239,1.000000e-02,1.000000e-02,0,3,1200,1.661992e+09,1.661992e+09,0.433884,-0.900969,0.087156,0.996195,0,0.000019,0.000001,0.000019,0.000001
2,10,10,2806.970000,12,2806.970000,12,5,0,12_8000F5030,12_8000F5030,2,2,2.806970e+03,2.806970e+03,0,3,120,1.661991e+09,1.661991e+09,0.433884,-0.900969,0.008727,0.999962,0,0.000195,0.000243,0.000195,0.000243
3,8,8,36682.970000,12,36682.970000,12,5,0,10_8000F5200,10_8000F5200,3,3,3.668297e+04,3.668297e+04,0,3,360,1.661991e+09,1.661991e+09,0.433884,-0.900969,0.026177,0.999657,0,0.000121,0.000008,0.000121,0.000008
4,0,0,6162.440000,12,6162.440000,12,5,0,1_8000F5AD0,1_8000F5AD0,4,4,6.162440e+03,6.162440e+03,0,3,180,1.661991e+09,1.661991e+09,0.433884,-0.900969,0.013090,0.999914,0,0.000177,0.000092,0.000177,0.000092
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250896,4222,3797,0.000223,1,0.000223,1,1,0,152627_813D6AFF1,53744_814237D21,48362,48634,1.876695e-08,1.876695e-08,23,5,85260,1.662853e+09,1.662853e+09,-0.974928,-0.222521,-0.082808,0.996566,1,0.000046,0.000007,0.000046,0.000007
1250897,4672,4671,0.417737,1,0.417737,1,1,0,254565_8147A3EC1,254355_8147A3E21,48834,75038,3.515537e-05,3.515537e-05,23,5,84720,1.662853e+09,1.662853e+09,-0.974928,-0.222521,-0.121869,0.992546,1,0.000037,0.000003,0.000037,0.000003
1250898,12,4222,0.051436,1,0.051436,1,1,0,15_813D8E211,152627_8147AA0B1,48375,69359,4.328685e-06,4.328685e-06,23,5,85860,1.662854e+09,1.662854e+09,-0.974928,-0.222521,-0.039260,0.999229,1,0.000046,0.000030,0.000046,0.000030
1250899,12,4222,0.084692,1,0.084692,1,1,0,15_813D8E211,152627_8147AA0B1,48375,69359,7.127401e-06,7.127401e-06,23,5,85200,1.662853e+09,1.662853e+09,-0.974928,-0.222521,-0.087156,0.996195,1,0.000046,0.000030,0.000046,0.000030


: 

In [ ]:
import pandas as pd

df = xgb_pipeline.df.copy()
df["TxID"] = df.index

###############################################################################
# 1. Prepare the "forward" lookup table, with from/to info for each transaction
###############################################################################
forward_df = (
    df[['TxID', 'from_account_id', 'to_account_id', 'sent_amount_usd']]
    .rename(columns={
        'TxID': 'forward_TxID',
        'from_account_id': 'fwd_from_account',
        'to_account_id': 'fwd_to_account',
        'sent_amount_usd': 'forward_amount'
    })
)

###############################################################################
# 2. 1-Hop Forward Merge
#    For each T1, find T2 where T2.fwd_from_account = T1.to_account_id
###############################################################################
df_forward = df.merge(
    forward_df,
    how='left',
    left_on='to_account_id',
    right_on='fwd_from_account'
)

# Remove self-joins: keep rows where forward_TxID is NaN or differs from T1’s TxID
df_forward = df_forward[
    df_forward['forward_TxID'].isna() | (df_forward['TxID'] != df_forward['forward_TxID'])
]

###############################################################################
# 3. 2-Hop Forward Merge
#    Now find T3 where T3.fwd_from_account = T2.fwd_to_account
###############################################################################
# First rename columns in forward_df to avoid collision in the second merge
forward_df_2 = forward_df.rename(columns={
    'forward_TxID': 'forward_TxID_2',
    'fwd_from_account': 'fwd_from_account_2',
    'fwd_to_account': 'fwd_to_account_2',
    'forward_amount': 'forward_amount_2'
})

df_forward2 = df_forward.merge(
    forward_df_2,
    how='left',
    left_on='fwd_to_account',
    right_on='fwd_from_account_2'
)

# Again, remove self-joins (T2 vs T3):
df_forward2 = df_forward2[
    df_forward2['forward_TxID_2'].isna() | (df_forward2['forward_TxID'] != df_forward2['forward_TxID_2'])
]

###############################################################################
# 4. Aggregate 2-Hop Forward Stats (grouped by the ORIGINAL TxID)
###############################################################################
fwd2_stats = (
    df_forward2
    .groupby('TxID')['forward_amount_2']
    .agg(['min', 'max', 'mean', 'std'])
    .add_prefix('twohop_forward_')
)

###############################################################################
# 5. Repeat for Backward (2 hops)
###############################################################################
backward_df = (
    df[['TxID', 'to_account_id', 'from_account_id', 'sent_amount_usd']]
    .rename(columns={
        'TxID': 'backward_TxID',
        'to_account_id': 'bwd_to_account',
        'from_account_id': 'bwd_from_account',
        'sent_amount_usd': 'backward_amount'
    })
)

# 1-Hop Backward
df_backward = df.merge(
    backward_df,
    how='left',
    left_on='from_account_id',
    right_on='bwd_to_account'
)
df_backward = df_backward[
    df_backward['backward_TxID'].isna() | (df_backward['TxID'] != df_backward['backward_TxID'])
]

# 2-Hop Backward
backward_df_2 = backward_df.rename(columns={
    'backward_TxID': 'backward_TxID_2',
    'bwd_to_account': 'bwd_to_account_2',
    'bwd_from_account': 'bwd_from_account_2',
    'backward_amount': 'backward_amount_2'
})
df_backward2 = df_backward.merge(
    backward_df_2,
    how='left',
    left_on='bwd_from_account',
    right_on='bwd_to_account_2'
)
df_backward2 = df_backward2[
    df_backward2['backward_TxID_2'].isna() | (df_backward2['backward_TxID'] != df_backward2['backward_TxID_2'])
]

# Aggregate 2-Hop Backward Stats
bwd2_stats = (
    df_backward2
    .groupby('TxID')['backward_amount_2']
    .agg(['min', 'max', 'mean', 'std'])
    .add_prefix('twohop_backward_')
)

###############################################################################
# 6. Merge 2-Hop stats back onto your final dataframe
###############################################################################
# Start with the 1-hop final if you already have it, or just start from df
df_final = df.set_index('TxID')

df_final = (
    df_final
    .join(fwd2_stats)      # 2-hop forward stats
    .join(bwd2_stats)      # 2-hop backward stats
    .reset_index()
)

df_final.head()


In [ ]:
df_final.to_csv('file.csv', index=False)

In [ ]:
xgb_pipeline.df = df_final.copy()

In [ ]:
X = ['received_amount', 'received_currency','sent_amount','sent_currency','payment_type','sent_amount_usd'
     ,'received_amount_usd','hour_of_day','day_of_week','seconds_since_midnight','timestamp_int','timestamp_scaled'
     ,'day_sin','day_cos','time_of_day_sin','time_of_day_cos','is_weekend','degree_centrality_sent_amount','pagerank_sent_amount'
     ,'degree_centrality_received_amount','pagerank_received_amount','onehop_forward_min', 'onehop_forward_max', 'onehop_forward_mean' 
     ,'onehop_forward_std', 'onehop_backward_min', 'onehop_backward_max', 'onehop_backward_mean', 'onehop_backward_std']

y = 'is_laundering'

X_train, X_val, X_test, y_train, y_val, y_test = xgb_pipeline.split_train_test_val(X,y)



In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score
import shap
import matplotlib.pyplot as plt

# Train XGBoost model
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='auc',
    use_label_encoder=False,
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

model.fit(X_train, y_train,
          early_stopping_rounds=10,
          eval_set=[(X_val, y_val)],
          verbose=True)

# Evaluate model on test set
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_prob):.4f}")


In [ ]:
# Train predictions
train_proba = model.predict_proba(X_train)[:, 1]  # Get probability of class 1
train_pred = (train_proba > 0.5).astype(int)

# Validation predictions
val_proba = model.predict_proba(X_val)[:, 1]
val_pred = (val_proba > 0.5).astype(int)

# Test predictions
test_proba = model.predict_proba(X_test)[:, 1]
test_pred = (test_proba > 0.5).astype(int)


# Evaluate Model Performance
slide_title = "Performance for XGBoost with New Features"
metrics_summary = xgb_pipeline.result_metrics(
    slide_title,
    y_train, train_pred, train_proba,
    y_val, val_pred, val_proba,
    y_test, test_pred, test_proba,
    class_labels=["Licit", "Illicit"]
)


In [ ]:

# SHAP Feature Importance
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_train, check_additivity=False)

# Summary Plot (Global Feature Importance)
shap.summary_plot(shap_values, X_train, plot_type="bar")

# Optional: Summary dot plot (per-sample contribution)
shap.summary_plot(shap_values, X_train)